In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from datetime import datetime
import distutils.dir_util
from keras.callbacks import CSVLogger
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, ZeroPadding2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D, AveragePooling2D, Activation, Flatten, Dropout
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator

vgg_base = VGG16(weights='imagenet', include_top=False)


Using TensorFlow backend.


In [2]:
classifier_input_shape = (1, 1, 2048)
classifier_input = Input(shape=classifier_input_shape)

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

#batch_size = 32

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/train',
        target_size=(224, 224),
        batch_size=28,
        #batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/valid',
        target_size=(224, 224),
        shuffle=False,
        #batch_size=batch_size,
        batch_size=20,
        class_mode='categorical')

# tried a bigger image size but was not good

Found 3276 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [4]:
def create_precomputed_data(model, batches):
    filenames = batches.filenames
    conv_features = model.predict_generator(batches, (batches.samples/batches.batch_size ), verbose=1)
    #conv_features = model.predict_generator(batches,
     #                              len(batches.filenames))
    labels = to_categorical(batches.classes)
    return (filenames, conv_features, labels)

In [5]:
trn_filenames, trn_conv_features, trn_labels = create_precomputed_data(vgg_base, train_generator)
val_filenames, val_conv_features, val_labels = create_precomputed_data(vgg_base, validation_generator)

25/25 [==============================] - 13s    


In [6]:
import matplotlib.pyplot as plt
trn_conv_features[0].shape


(7, 7, 512)

In [7]:
trn_conv_features.shape

classifier_input_shape = (7, 7, 512)
classifier_input = Input(shape=classifier_input_shape)

In [8]:
p = 0.6
conv_layers = vgg_base.layers

In [9]:


def get_bn_layers(p):
    return [
#         Flatten()(Input(input_shape=conv_layers[-1].output_shape[1:])),
        MaxPooling2D(Input(input_shape=conv_layers[-1].output_shape[1:])),
        BatchNormalization(axis=1),
        Dropout(p/4),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(8, activation='softmax')
    ]

In [10]:
nf=128; p=0.5

In [11]:

def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=classifier_input_shape),
        Conv2D(nf,(3,3), activation='relu', padding='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(nf,(3,3), activation='relu', padding='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(nf,(3,3), activation='relu', padding='same'),
        BatchNormalization(axis=1),
        Conv2D(nf,(3,3), activation='relu', padding='same'),
        BatchNormalization(axis=1),
#         MaxPooling2D((1,2)),
        Conv2D(8,(3,3), padding='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [12]:
# bn_model = Sequential(get_bn_layers(p))
lrg_model = Sequential(get_lrg_layers())

In [13]:
lrg_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
lrg_model.fit(trn_conv_features, trn_labels, batch_size=32, epochs=2, 
             validation_data=(val_conv_features, val_labels))

Train on 3276 samples, validate on 500 samples
Epoch 1/2
3276/3276 [==============================] - 17s - loss: 0.9092 - acc: 0.7057 - val_loss: 0.3317 - val_acc: 0.9000
Epoch 2/2
3276/3276 [==============================] - 17s - loss: 0.8175 - acc: 0.7372 - val_loss: 0.3759 - val_acc: 0.8880


In [31]:
lrg_model.optimizer.lr = 0.01

In [18]:
lrg_model.load_weights('/home/ubuntu/git/learningWithKaggle/weights/fishing/ft_vggft_91_valid.h5')

## make a submission

In [20]:
train_generator.class_indices

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

In [21]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = validation_datagen.flow_from_directory(
        '/home/ubuntu/data/fishing/test_stg1',
        target_size=(224, 224),
        shuffle=False,
        batch_size=20,
        class_mode=None)

Found 1000 images belonging to 1 classes.


In [22]:
batches = test_generator

test_preds = lrg_model.predict_generator(batches, (batches.samples/batches.batch_size ), verbose=1)

ValueError: Error when checking : expected batch_normalization_1_input to have shape (None, 7, 7, 512) but got array with shape (20, 224, 224, 3)